# [concurrent.futures 模块提供异步执行可调用对象高层接口。](https://docs.python.org/zh-cn/3/library/concurrent.futures.html#module-concurrent.futures)

##  异步执行可以由 ThreadPoolExecutor 使用线程或由 ProcessPoolExecutor 使用单独的进程来实现。 两者都是实现抽像类 Executor 定义的接口。

In [1]:
%%writefile test_thread.py
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor
import os,time,random

def task(name,n):
    print(f"{name} is running, pid is {os.getpid()}")
    time.sleep(random.randint(1,3))
    return n*n

if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=5)
    
    for i in range(10):
        future=pool.submit(task,f"egon{i}",i)
        print(f"egon{i}'s result is {future.result()}") #* 如果注销该行，会直接异步提交下一个，否则会等到有结果才会继续下一个提交
    
    print('主')
        

Overwriting test_thread.py


## shutdown(wait=True, *, cancel_futures=False)
相当于进程池的pool.close()+pool.join()操作

wait=True，等待池内所有任务执行完毕回收完资源后才继续

wait=False，立即返回，并不会等待池内的任务执行完毕

但不管wait参数为何值，整个程序都会等到所有任务执行完毕

submit和map必须在shutdown之前

In [2]:
%%writefile test_thread.py
from concurrent.futures import ProcessPoolExecutor
import os,time,random

def task(name):
    print(f"{name} is running, pid is {os.getpid()}")
    time.sleep(random.randint(1,3))

if __name__ == "__main__":
    pool=ProcessPoolExecutor(max_workers=5)
    
    for i in range(10):
        pool.submit(task,f"egon{i}")
        
    # 如果 wait 为 True 则此方法只有在所有待执行的 future 对象完成执行且释放已分配的资源后才会返回。 如果 wait 为 False，方法立即返回，所有待执行的 future 对象完成执行后会释放已分配的资源。 不管 wait 的值是什么，整个 Python 程序将等到所有待执行的 future 对象完成执行后才退出。
    pool.shutdown(wait=True) #* 如果使用 with 语句，你就可以避免显式调用这个方法，它将会停止 Executor (就好像 Executor.shutdown() 调用时 wait 设为 True 一样等待):
    
    #* 上式等效于
    # with ProcessPoolExecutor(max_workers=5) as executor:
    #     for i in range(10):
    #         executor.submit(task,f"egon{i}")
    
    
    print('主')
        

Overwriting test_thread.py


## map(func, *iterables, timeout=None, chunksize=1)

In [3]:
%%writefile test_thread.py
from concurrent.futures import ThreadPoolExecutor
from threading import currentThread
import os,time,random

def task():
    print(f"{currentThread().getName()} is running, pid is {os.getpid()}")
    time.sleep(random.randint(1,3))

if __name__ == "__main__":
   with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(task,range(1,12))

    print('主')

Overwriting test_thread.py
